# Using Numerical Intergration Methods to Calculate Constants and Uncertainty Values in Quantum Mechanics

# Introduction

This report will focus on the calculation of expectation values of position or momentum and their respective uncertainties and finding the values of unknown constants in the wavefunctions using the normalisation conditions. 


We will consider a 1D finite potential well, the general form of the wavefunction is known in each region however each wave function has an unknown constant which can be found using the normalisation condition (eq 1). Once we have found the value of these unkown constants the expectation values can be calculated using equation 2. These values can then be used to find the uncertainties in position (eq 3). The product of the uncertainties must be greater than $\frac{\hbar}{2}$ (eq 4), this can be used to approximate the uncertainty in momentum.  


**Equations**

1. Normalisation Condition $\int_{-\infty}^{+\infty}{(\Psi^{*}\Psi) dx} = 1$

2. Expectation Value of x $ <x> = \int_{-\infty}^{+\infty}{(\Psi^{*}x\Psi) dx}$  $\space \space \large $ 

3. Uncertainty in x $ \Delta x = \sqrt{<x^{2}> - <x>^{2}}$

4. Uncertainty Relation $\Delta p .\Delta x\geq \frac{\hbar}{2}$

# Method

The methods of numerical integration we are going to use in this reports are the Trapezium rule, Simpson's rule and Monte-Carlo integration. The `quad` function from `scipy.integrate` will also be used for comparative purposes. More details about each numerical integration method will be located under their own specific section.   

Each function will be coded, then tested using a cosine and exponential function. The values of the integral calclulated for these functions are well known and will indicate any issues in the coded functions. After testing the functions can be apllied. 


# Coding each Numerical integration Method

In [1]:
import numpy as np
from scipy.integrate import quad
from scipy import random
from scipy.special import factorial as fac
import matplotlib.pyplot as plt

### Test Functions

The test functions:

1. $f(x) = cos(x)$
2. $f(x) = e^{-x}$

We know that $\int^{\frac{\pi}{2}}_{0}{cos(x)}dx = 1$ and $\int^{+\infty}_{0}{e^{-x}}dx = 1$.

In [2]:
def cos(x):
    return np.cos(x)

def e(x):
    return np.exp(-x)

maxim = 1000

### Trapezium and Simpson Rule

The Trapezium Rule and the Simpson Rule are two simple methods of integration. Both methods are very similar wth the expection of thier weighting. The function below works for both trapezium and simpson. One of the function parameters is a string stating the type of method required which can either be `'sim'`(for simpsons method) or `'trap'`(for trapezuim method). The function contains a branch and will weight respectively. If the bounds entered are improper the function tries to create an `x` array of infinite length. This will only affect the wavefunction outside the well as the wavefunction exponentially decays. 


The weighting arrays $V$ for the trapezium and Simpson rule are:   

1. $V_{Trapezium} = (\tfrac12, 1, 1, \dots, 1, \tfrac12)$

2. $V_{Simpson} = \tfrac13\times(1, 4, 2, 4, 2, \dots, 4, 2, 4, 1)$


In [3]:
def trapson(function, step, bounds, method):
    """Integrates using the trapezium rule or the simpson rule. 
       Inputs are a function of x, the step size and the bounds, a tuple in the format bounds = (lower, upper)
       enter method as string: either 'sim for simpson or 'trap' for trapezium """
     
    lb, ub = bounds
    
    if ub == np.inf:
        ub = maxim
        
    if lb == -np.inf:
        lb = -maxim
    
    if ub == -np.inf:
        ub = -maxim
        
    if lb == np.inf:
        lb = maxim
         
    x = np.linspace(lb, ub, (abs(ub - lb) / step))
    y = function(x)
    
    if method == 'trap':
        v = np.ones(len(x))
        v[0] = (1 / 2)
        v[-1] = (1 / 2)
    
    elif method == 'sim':
        v = np.ones([len(x)])
        v[::2]=2
        v[1::2]=4
        v[0]=1
        v[-1]=1
        v = v / 3
    
    I = step * np.sum(y * v)
    
    return abs(I)

***Testing the Simpson and Trapezium methods***

In [4]:
t1 = trapson(e, 0.01, (0, np.inf), 'trap')
t2 = trapson(e, 0.01, (0, np.inf), 'sim')


print(t1, 'error =', abs(1 - t1))
%timeit t1 = trapson(e, 0.01, (0, np.inf), 'trap')
print(t2,'error =', abs(1 - t2))
%timeit t2 = trapson(e, 0.01, (0, np.inf), 'sim')

C:\Users\patel\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


0.9999983334027783 error = 1.6665972216722835e-06
10.3 ms ± 717 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
0.9999900000555566 error = 9.999944443395137e-06
9.01 ms ± 358 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
t3 = trapson(cos, 0.01, (0, np.pi / 2), 'trap')
t4 = trapson(cos, 0.01, (0, np.pi / 2), 'sim')

print(t3, 'error =', abs(1 - t3))
%timeit trapson(cos, 0.01, (0, np.pi / 2), 'trap')
print(t4,'error =', abs(1 - t4))
%timeit trapson(cos, 0.01, (0, np.pi / 2), 'sim')

C:\Users\patel\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


0.9931184538732284 error = 0.006881546126771609
49.9 µs ± 1.94 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
0.9931268449501447 error = 0.006873155049855306
73.6 µs ± 26.9 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Monte-Carlo Integration

This methos of integration uses a sample of random numbers along with the formula for the average value of a function in order to approximate the value of an integral. This method is only precise if ran repeatedly, furthermore a normal distribution can be viewed to determine the modal value. Accuracy can be increased by increasing the number of runs.

The average value of a function, $f$, is

$\large \overline{f} = \frac{1}{b-a}\int_a^b f(x)\;\mathrm{dx}$

by rearranging we obtain:

$\large\overline{f}\times\frac{1}{b-a}$   $=    \int_a^b f(x)\;\mathrm{dx}$

$\overline{f}$ can then be defined as $\overline{f}$ = $(b-a)\frac{1}{N}\sum_{i}^{N}{f(x_{i})}$ giving:

$\large\int_a^b f(x)\;\mathrm{dx}$ = $(b-a)\frac{1}{N}\sum_{i}^{N}{f(x_{i})}$ $[1]$



In [6]:
def mc(function, bounds, no_runs):
    """Returns the value of an integral using the Monte-Carlo method. 
        Inputs are a function, the bounds, a tuple in the form (lower, upper) and no_runs, 
        the number of times you wish to integrate and average. 1000 to 1500 is ideal."""
    
    lb, ub = bounds
    
    if ub == np.inf:
        ub = maxim
        
    if lb == -np.inf:
        lb = -maxim
    
    if ub == -np.inf:
        ub = -maxim
        
    if lb == np.inf:
        lb = maxim
            
    
    def integrate(a, b, N):
        
        ran = random.uniform(a, b, N)

        I = 0.0

        for j in range(no_runs):
                I += function(ran[j])
    

        result = ((b - a) / float(N)) * I
        return result
    
    area = []
    
    for i in range(no_runs):
        
        I = integrate(lb, ub, no_runs)
        area.append(I)
    
    area = np.array(area)
    
    average = np.sum(area) / len(area)
    
    return average
    

In [7]:
t5 = mc(e, (0, np.inf), 1000)
%timeit mc(e, (0, np.inf), 1000)
print(t5, 'error =', abs(1 - t5))

1.67 s ± 23.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.0206359310383695 error = 0.02063593103836947


In [8]:
t6 = mc(cos, (0, np.pi / 2), 1000)
%timeit mc(cos, (0, np.pi / 2), 1000)
print(t6, 'error =', abs(1 - t6))

2.33 s ± 398 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.0006280278758999 error = 0.0006280278758998836


### Evaluating Results from Test Functions

Two simple functions used were:
1. $f_1(x) = cos(x)$ 
2. $f_2(x) = e^{-x}$

The expected values are $\int^{+\infty}_{0}{e^{-x}}dx = 1$ and $\int^{\frac{\pi}{2}}_{0}{cos(x)}dx = 1$

$f_1(x)$

|Integration Method|Limits of Integration $(l, u)$ |Value Obtained| Error |Percentage Error %| Time Taken|
|----|----|----|----||  |
|Trapezium |$(0, \frac{\pi}{2})$  |0.9931184538732284|0.006881546126771609|0.68815461267|49.9 µs ± 1.94 µs per loop|
|Simpson|$(0, \frac{\pi}{2})$|0.9931268449501447|0.006873155049855306|0.68731550498|73.6 µs ± 26.9 µs per loop|
|Montecarlo|$(0, \frac{\pi}{2})$|1.0006280278758999|0.0006280278758998836|0.06280278759|2.33 s ± 398 ms per loop|


$f_2(x)$

|Integration Method|Limits of Integration $(l, u)$ |Value Obtained| Error |Percentage Error %| Time Taken|
|----|----|----|----||  |
|Trapezium |$(0, \infty)$|0.9999983334027783|1.6665972216722835e-06|0.00016665972|10.3 ms ± 717 µs per loop|
|Simpson|$(0, \infty)$|0.9931268449501447|9.999944443395137e-06|0.68731550498|9.01 ms ± 358 µs per loop|
|Montecarlo|$(0, \infty)$|1.0206359310383695|0.02063593103836947|2.06359310384|1.67 s ± 23.6 ms per loop|

These results show that the accuracy of the results can vary depending on what type of function we are trying to integrate. 
We cansee for the cos function the Monte-Carlo method was the most accurate but it took the longest amount of time to produce a result. For the exponential function the trapezuim method prodices the most accurate result. 

We can see that the Monte-Carlo method takes the most amount of tmie to produce a result compared to the trapezuim and simpson methads. 

## Application of Numerical Integration Methods to a 1D Problem

The objective is to determine the value of the constants and then use them to find the expectation values of the position and momentum and then the uncertainties in position and momentum and the probability of finding a particle in the well. 

From the the Schrödinger equation, we know the general form of the wavefunction for each region. In this example the particle is an electron ($m_{e} = 0.51099895000 MeV/c^{2}$), the depth of the well, $V_{0} = 14eV$, the width of the well, $L = 4Å = 4 \times 10^{-10}m$ and the electron is in the ground state of $1.47eV.$ $\hbar^{2} = 6.5821\times 10^{-16}eVs.$

$k = \sqrt{\frac{2mE}{\hbar^{2}}}$ and $\kappa = \sqrt{\frac{2m(V_{0}-E)}{\hbar^{2}}}$  

Region 1 (outside well left): $\psi_{1}(x) = Be^{\kappa x}$

Region 2 (inside well): $\psi_{2}(x) = Ccos(kx)$  $\space \space \space \space\large$

Region 3 (outside well right): $\psi_{3}(x) = Ae^{-\kappa x}$ 

The normalisation condition states:

$\int_{-\infty}^{+\infty}{|\Psi^{2}|} dx = 1$ = $\int_{-\infty}^{+\infty}{(\Psi^{*}\Psi) dx}$ 

The wavefunction is not complex in this case so $\Psi^{*}\Psi = \Psi^{2}$

$\Psi(x)$ can then be inserted into the normalisation formula we can rearrange to find $B^{2}, A^{2}$ and $C^{2}$

$B^{2} = \Large\frac{1}{\int_{-\infty}^{\small\frac{-L}{2}}{(e^{\kappa x})^{2}}}$, $C^{2} = \Large\frac{1}{\int_{\small\frac{-L}{2}}^{\small\frac{L}{2}}{cos^{2}(x)}}$ and $A^{2} = \Large\frac{1}{\int_{\small\frac{L}{2}}^{+\infty}{(e^{-\kappa x})^{2}}}$  $\space \space \large [3]$

We will be using SI units for better consistency. These variables will be used to represent each region -- r1 (region 1), r2 (regions 2) and r3 (region 3) 

In [9]:
m = 9.11e-31
V0 = 14 * 1.6e-19
E = 1.47 * 1.6e-19
L = 4e-10
hbar = 6.626e-34 / (2 * np.pi)

k = np.sqrt((2 * m * E) / (hbar ** 2))
kappa = np.sqrt((2 * m * (V0 - E)) / (hbar ** 2))


def r1(x):
    return np.exp(kappa * x) ** 2 


def r2(x):
    return np.cos(k * x) ** 2 


def r3(x):
    return np.exp(-kappa * x) ** 2  

### Finding the coefficients

Find B, C and A from:

Region 1 (outside well left): $\psi_{1}(x) = Be^{\kappa x}$

Region 2 (inside well): $\psi_{2}(x) = Ccos(kx)$  $\space \space \space \space\large$

Region 3 (outside well right): $\psi_{3}(x) = Ae^{-\kappa x}$ 


***Region 2 (inside well) find coefficient C***

In [10]:
inwellstep = 1e-16    
outwellstep = 1e-9

In [11]:
C_trap = np.sqrt(1 / trapson(r2, inwellstep, (-L / 2, L / 2), 'trap'))  
print('Trapezium method:', C_trap)

C_sim = np.sqrt(1 / trapson(r2, inwellstep, (-L / 2, L / 2), 'sim'))
print('Simpson method:', C_sim)

C_mc = np.sqrt(1 / mc(r2, (-L / 2, L / 2), 2500))
print('Monte-Carlo method:', C_mc)

Cq = np.sqrt(1 / quad(r2,-L / 2, L / 2)[0])
print(Cq)

C:\Users\patel\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


Trapezium method: 63335.130824578344
Simpson method: 63335.13126733856
Monte-Carlo method: 63332.77147652923
63335.122907685705


C will be taken to be 6335

***Region 1 (outside well left) find coefficient B***

In [12]:
B_trap = np.sqrt(1 / trapson(r1, 0.01, (-np.inf, -L / 2), 'trap'))
print('Trapezium method:', B_trap)

B_sim = np.sqrt(1 / trapson(r1, 0.01, (-np.inf, -L / 2), 'sim'))
print('Simpson method:', B_sim)

B_mc = np.sqrt(1 / mc(r1, (-np.inf, -L / 2), 2500))
print('Monte-Carlo method:', B_mc)

print(quad(r1, -np.inf, -L/2))

C:\Users\patel\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


Trapezium method: 530.5031777482387
Simpson method: 649.7310462040961
Monte-Carlo method: inf
(0.0, 0.0)


C:\Users\patel\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


Values differ, no conclusive answer.

***Region 3 (outside well right) find coefficient A***

In [13]:
A_trap = np.sqrt(1 / trapson(r3, 0.01,(L / 2, np.inf), 'trap'))
print('Trapezium method:', A_trap)

A_sim = trapson(r3, 0.01,(L / 2, np.inf), 'sim')
print('Simpson method:', A_sim)

A_mc = np.sqrt(1 / mc(r3, (L / 2, np.inf), 2500))
print('Monte-Carlo method:', A_mc)

quad(r3, L / 2, 3e-8)[0], quad(r3, L / 2, np.inf)[0]

C:\Users\patel\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


Trapezium method: 530.5031777482387
Simpson method: 2.3688238202465947e-06
Monte-Carlo method: inf


C:\Users\patel\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


(1.9314124953958087e-14, 0.0)

No conclusive answer

### Calculating B and A Using Boundary Conditions

The wavefunction in region 1 and region 3 meet the wavefunction in region 2 at $-\frac{L}{2}$ and $\frac{L}{2}$ respectively. 

After rearranging we obtain:

1. $Be^{\kappa x} = Ccos(-kx) \rightarrow    B = Ccos(kx)e^{-\kappa x}$ at $x = -\frac{L}{2}$


2. $Ae^{-\kappa x} = Ccos(kx)  \rightarrow    A = Ccos(kx)e^{\kappa x}$ at $x = \frac{L}{2}$

***Finding B and A***

In [14]:
C = 63335

B = C * np.cos(k * (-L / 2)) * np.exp(-kappa * (-L / 2))
print('B = ', B)

A = C * np.cos(k * (L / 2)) * np.exp(kappa * (L / 2))
print('A = ', A)

B =  768262.576489645
A =  768262.576489645


In [15]:
A == B

True

A = B = 768262.576489645

We can take the value for A and B to be approximately 768262

The wavefunctions for this finite square well have been obtained and are the following:

Region 1: $\psi_{1}(x) = 768263e^{\kappa x}$

Region 2: $\psi_{2}(x) = 63335cos(kx)$

Region 3: $\psi_{3}(x) = 768263e^{-\kappa x}$

The problem of integrating with infinite bounds must now be adressed. For the function $ f(x) = e^{x}$, or $ g(x) = e^{-x}$, $f(-710) = 0$ and $g(710) = \infty$. Taking $\pm709$ ($e^{709}$ has a finite value) as the limits of the total exponent, the region of x we can integrate over has to be equal to this limit. Regarding the two exponential wavefunctions, $\kappa x \leq 709$ and $-\kappa x \geq -709$. Therefore:

$x_{max} = \frac{709}{\kappa}$

$x_{min} = -\frac{709}{\kappa}$

We will use trial and error until the exponential wavefunctions stop returning a zero value.

In [16]:
def psi_tr1(x):
    psi = (B * np.exp(kappa * x))   
    return psi


def psi_tr2(x):
    psi = C * np.cos(k * x)          
    return psi


def psi_tr3(x):
    psi = A * np.exp(-kappa * x)        
    return psi

In [17]:
np.exp(709.5), np.exp(710)

(1.3549863193146328e+308, inf)

In [18]:
xmax = 372 / kappa
xmin = -372 / kappa
infbounds = (xmin,xmax) 
np.sqrt(1 / trapson(r3, outwellstep,(L / 2, infbounds[1]), 'trap'))

C:\Users\patel\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


1677598.3476415905

In [19]:
psi_tr1(infbounds[0]), psi_tr3(infbounds[1])

(2.1279548565743146e-156, 2.1279548565743146e-156)

372 is the limit before the wavefunctions return a zero value.

### Calculating the Expectation Values for Position of Electron.

Expectation value of x is given by:

$<x> = \int_{-\infty}^{+\infty}{(\Psi^{*}x\Psi) dx}$

and $<x^{2}> = \int_{-\infty}^{+\infty}{(\Psi^{*}x^{2}\Psi) dx}$


The functions bellow return values of $\Psi^{*}x\Psi$ and $\Psi^{*}x^{2}\Psi$ .

In [20]:
A = 768263
B = 768263
C = 63335


def psi_ev_r1(x):
    psi = (B * np.exp(kappa * x))   
    return psi * psi * x

def psi_ev_sqr1(x):
    psi = (B * np.exp(kappa * x))   
    return psi * psi * x * x


def psi_ev_r2(x):
    psi = C * np.cos(k * x)          
    return psi * psi * x

def psi_ev_sqr2(x):
    psi = C * np.cos(k * x)          
    return psi * psi * x * x


def psi_ev_r3(x):
    psi = A * np.exp(-kappa * x)       
    return psi * psi * x

def psi_ev_sqr3(x):
    psi = A * np.exp(-kappa * x)        
    return psi * psi * x * x

***Region 2 Expectation Values***

In [28]:
ev_trap_r2 = trapson(psi_ev_r2, inwellstep , (-L / 2, L / 2), 'trap')
print('Trapezium Method = ', ev_trap_r2)

ev_sim_r2 = trapson(psi_ev_r2, inwellstep , (-L / 2, L / 2), 'sim')
print('Simpson Method = ', ev_sim_r2)

ev_mc_r2 = mc(psi_ev_r2, (-L / 2, L / 2), 1000)
print('Monte-Carlo = ', ev_mc_r2)

ev_q2 = quad(psi_ev_r2, -L / 2, L / 2)
print(ev_q2)

C:\Users\patel\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


Trapezium Method =  6.832578947069124e-27
Simpson Method =  2.796293726669319e-18
Monte-Carlo =  8.452903539025018e-14
(0.0, 8.056413590118503e-25)


***Region 1 Expectation Values***

In [36]:
ev_trap_r1 = trapson(psi_ev_r1, outwellstep, (infbounds[0], -L / 2), 'trap')
print('Trapezium Method = ', ev_trap_r1)

ev_sim_r1 = trapson(psi_ev_r1, outwellstep, (infbounds[0], -L / 2), 'sim')
print('Simpson Method = ', ev_sim_r1)

ev_mc_r1 = mc(psi_ev_r1, (infbounds[0], -L / 2), 1000)
print('Monte-Carlo = ', ev_mc_r1)

ev_q1 = quad(psi_ev_r1, infbounds[0], -L / 2)
print(ev_q1)

C:\Users\patel\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


Trapezium Method =  4.19443870146996e-11
Simpson Method =  2.7962924676466396e-11
Monte-Carlo =  -2.703195413304319e-12
(-2.6763744341322383e-12, 5.233976426815831e-12)


***Region 3 Expectation Values***

In [34]:
ev_trap_r3 = trapson(psi_ev_r3, outwellstep , (L / 2, infbounds[1]), 'trap')
print('Trapezium Method = ', ev_trap_r3)

ev_sim_r3 = trapson(psi_ev_r3, outwellstep, (L / 2, infbounds[1]), 'sim')
print('Simpson Method = ', ev_sim_r3)

ev_mc_r3 = mc(psi_ev_r3, (L / 2, infbounds[1]), 1000)
print('Monte-Carlo = ', ev_mc_r3)

ev_q3 = quad(psi_ev_r3, L / 2, infbounds[1])
print(ev_q3)

C:\Users\patel\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


Trapezium Method =  4.19443870146996e-11
Simpson Method =  2.7962924676466403e-11
Monte-Carlo =  2.5061502102774888e-12
(2.6763744341322383e-12, 5.233976426815831e-12)


Results produced by the Monte-Carlo Integration Method will be used to work out uncertainties. Due to Monte-Carlo Integration Method and Quad producing results of the same order of magnitude in region 1 and 3. 

### Calculating Uncertainty in Position ($\Delta x$)

The formula for the uncertainty in position is:

Uncertainty in x, $ \Delta x = \sqrt{<x^{2}> - <x>^{2}}$


In [37]:
def uncertainty(expectation, expectation_sq):
    """ Returns the Uncertainty value from two expectation values."""
    
    delta = np.sqrt(expectation_sq - expectation**2)
    return delta

***Region 2***

In [38]:
ev_sq_r2 = mc(psi_ev_sqr2, (-L / 2, L / 2), 1000)
print('<x^2> =', ev_sq_r2)

Dx_r2 = uncertainty(ev_mc_r2, ev_sq_r2)
print('Δx =', Dx_r2)

<x^2> = 7.80397535551574e-21
Δx = 8.834007137396888e-11


***Region 1***

In [39]:
ev_sq_r1 = mc(psi_ev_sqr1, (infbounds[0], -L / 2), 1000)
print('<x^2> =', ev_sq_r1)

Dx_r1 = uncertainty(ev_mc_r1, ev_sq_r1)
print('Δx =', Dx_r1)

<x^2> = 6.119703641497254e-22
Δx = 2.4589898306158484e-11


***Region 3***

In [40]:
ev_sq_r3 = mc(psi_ev_sqr3, (L / 2, infbounds[1]), 1000)
print('<x^2> =', ev_sq_r3)

Dx_r3 = uncertainty(ev_mc_r3, ev_sq_r3)
print('Δx =', Dx_r3)

<x^2> = 6.03253296608567e-22
Δx = 2.4433020847453412e-11


All the values of uncertainties in position have the same order of magnitude but differing values.

### Estimating the uncertainty in momentum

Heisenberg's uncertainty relations state that:

$\Delta x . \Delta p \geq \frac{\hbar}{2}$

$\Delta p \geq \frac{\hbar}{2 \Delta x}$

In [41]:
def momentum(delta_x):
    """ Returns an estimate for the uncertainty in momentum."""
    
    D_p = hbar / (2 * delta_x)
    
    return D_p

In [42]:
D_p_r1 = momentum(Dx_r1)
D_p_r2 = momentum(Dx_r2)
D_p_r3 = momentum(Dx_r3)

print (D_p_r1, D_p_r2, D_p_r3)

2.1442964907723635e-24 5.968755948037672e-25 2.1580644069986387e-24


Estimations of uncertainties in momentum are:

$\Delta p_{r1} \geq 2.144 \times 10^{-24} $

$\Delta p_{r2} \geq 5.968 \times 10^{-25} $

$\Delta p_{r3} \geq 2.158 \times 10^{-24} $

These uncertainty values show the lowest possible value of the uncertaint (does not give an exact value for the momentum).

## Results

### Unkown Constsnts

The integration techniques developed were used to find the unknown constant C for the wavefunction inside the finite potential well, the constants A and B were not found using the same method but were evaluated by using the wavefunctions at the boundary conditions. This faliure of the code was due to problems processing very large and very small numbers and the structure of the algorithm. 

The values of C obtained are:

| Value of $C$  | Method used |
|----|----|
|63335.130824578344|Trapezium|
|63335.13126733856|Simpson |
|63332.77147652923|Monte-Carlo|
|63335.122907685705|Quad -- `Scipy` |


The values of A and B were found using boundary conditions and were found to be the same. A = B = 768262.576489645 and we took the value of A and B to be 768263. 

### Uncertainty in Position

From the equations 2, 3 and 4 the methods of numerical integration were used. Simpson and trapezium methods were not able to calculate expectation values to the same order as the Monte-Carlo and Quad methods. I decided to use the Monte-Carlo values for the calculation of uncertainty in position.  

The uncertainty values are:

|  Region |Uncertainty in Position $\Delta x$  |
|----|----|
|1|2.4589898306158484$\times 10^{-11}$|
|2|8.834007137396888$\times 10^{-11}$|
|3|2.4433020847453412$\times 10^{-11}$|


The results show the uncertainties are all of the same order but are larger inside the well. The reason for this is there are a grater number of loctions the electron could possibly be in thuss the greater uncertainty.

### Uncertainty in Momentum 

Using these values of uncertainty in position and the Heisenberg Uncertainty Relation (eq 4), the lowest amount of uncertainty in momentum was found. 
$\space \space \Delta p .\Delta x\geq \frac{\hbar}{2}$

|  Region |Uncertainty in Momentum $\Delta p$  |
|----|----|
|1|$\Delta p_{r1} \geq 2.144 \times 10^{-24} $|
|2|$\Delta p_{r2} \geq 5.968 \times 10^{-25} $|
|3|$\Delta p_{r3} \geq 2.158 \times 10^{-24} $|


Region 2 has a lower uncertainty in mometum than regions 1 and 3 this is due to the fact that region 2 has a higher uncertainty in posotion than the other two regions. 

These values give us an idea of the lowest value of $\Delta p$ but is not nessacarily accurate. the value are showing the lower limit of $\Delta p$. 

## Conclusion

From our results we can see that the trapezuim method produced good approximation for integrals of exponentially decaying parts of wavefunctions for areas outside outside the well. The Monte-Carlo integration method provided more accurate values for trigonometric functions for the areas inside the well. We obtained more accurate approximations by dividing the wavefunctions into three distinct part and choose the best method of numerical integration for each part. 

This report shows that the simple methods of numerical integration can be adapted to solve more complicated wavefunctions (which are composed of non-complex and simple fuction) to a good level off accuracy.   


## References 

[1] https://qmplus.qmul.ac.uk/pluginfile.php/845236/mod_resource/content/3/Finite_well_tex.pdf

[2] https://www.wolframalpha.com/input/?i=integrate%20cos%28x%29dx%20from%200%20to%20pi%2F2

[3] https://www.wolframalpha.com/input/?i=integrate+e%5E-x+from+0+to+inf

[4] https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html

[5] Modern Physics 3rd Edition, Kenneth S. Krane, Wiley Publishing, Chapeters 4 and 5 
